In [8]:
import numpy as np 
import pandas as pd 
import polars as pl
import os
import optuna
import re
import warnings
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm import log_evaluation, early_stopping

warnings.simplefilter('ignore')

In [9]:
train = pd.read_csv('/home/baebro/kaggle_ws/car price/data/train.csv')
test = pd.read_csv('/home/baebro/kaggle_ws/car price/data/test.csv')
orig = pd.read_csv('/home/baebro/kaggle_ws/car price/data/used_cars.csv')


orig[['milage', 'price']] = orig[['milage', 'price']].apply(
    lambda col: col.map(lambda x: int(''.join(re.findall(r'\d+', str(x))))))

train = pd.concat([train, orig], ignore_index=True)

In [10]:
def fill_nan_values(df):
    
    freq = 100
    
    categoricl_colums = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title']
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for column in re_:
        df.loc[df[column].value_counts(dropna=False)[df[column]].values < freq, column] = "noise"
        
    for column in categoricl_colums:
        df[column] = df[column].fillna('missing')
        df[column] = df[column].astype('category')
        
    return df

train  = fill_nan_values(train)
test   = fill_nan_values(test)

In [11]:
X_train = train.drop(columns=["id", "price"])
y_train = train["price"]
X_test = test.drop(columns=["id"])

In [12]:
def cross_validate_model_l(model, X_train, y_train, X_test, params, n_splits=5):

    callbacks = [log_evaluation(period=150), early_stopping(stopping_rounds=200)]
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    val_scores = []
    test_preds = np.zeros((len(X_test), n_splits), dtype=np.float32)

    for fold, (train_ind, valid_ind) in enumerate(cv.split(X_train)):
        # Data splitting
        X_fold_train = X_train.iloc[train_ind]
        y_fold_train = y_train.iloc[train_ind]
        X_val = X_train.iloc[valid_ind]
        y_val = y_train.iloc[valid_ind]
        
        clf = model(**params)
        clf.fit(X_fold_train, y_fold_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
        
        test_preds[:, fold] = clf.predict(X_test)

        print("-" * 50)
        print(f"Fold {fold + 1} predictions: {test_preds[:, fold]}")

    test_preds = np.mean(test_preds, axis=1)
    return clf, test_preds

In [13]:
X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.1),
        'n_estimators': 10000,
        'metric': 'rmse',
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'random_state': 42
    }

    clf, test_preds = cross_validate_model_l(LGBMRegressor, X_train_full, y_train_full, X_test, params)
    
    y_val_pred = clf.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    return rmse

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best hyperparameters
print(study.best_params)

[I 2024-09-12 20:34:11,896] A new study created in memory with name: no-name-897d5287-fef8-42d6-bc82-9aa7c65f114d


[LightGBM] [Warning] bagging_fraction is set=0.7325484018132415, subsample=0.7684062337713805 will be ignored. Current value: bagging_fraction=0.7325484018132415
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=65, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=65
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 77946.5
[300]	valid_0's rmse: 77032.4
[450]	valid_0's rmse: 76277.5
[600]	valid_0's rmse: 75642.9
[750]	valid_0's rmse: 75118.5
[900]	valid_0's rmse: 74682.4
[1050]	valid_0's rmse: 74325.2
[1200]	valid_0's rmse: 74034.7
[1350]	valid_0's rmse: 73792.7
[1500]	valid_0's rmse: 73595.3
[1650]	valid_0's rmse: 73437
[1800]	valid_0's rmse: 73308.8
[1950]	valid_0's rmse: 73204.4
[2100]	valid_0's rmse: 73121
[2250]	valid_0's rmse: 73053.5
[2400]	valid_0's rmse: 73002.6
[2550]	valid_0's rmse: 72961.9
[2700]	valid_0's rmse: 72929.

[I 2024-09-12 20:42:55,320] Trial 0 finished with value: 75341.25196857267 and parameters: {'num_leaves': 309, 'max_depth': 11, 'learning_rate': 0.0007299168345403824, 'subsample': 0.7684062337713805, 'colsample_bytree': 0.7006328226700693, 'reg_alpha': 8.685848861466111e-06, 'reg_lambda': 4.15630023846269e-05, 'min_data_in_leaf': 65, 'bagging_fraction': 0.7325484018132415, 'bagging_freq': 3}. Best is trial 0 with value: 75341.25196857267.


[LightGBM] [Warning] bagging_fraction is set=0.5613792799670121, subsample=0.626110627013553 will be ignored. Current value: bagging_fraction=0.5613792799670121
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 74176.9
[300]	valid_0's rmse: 73159.3
[450]	valid_0's rmse: 72997.4
[600]	valid_0's rmse: 73006.6
Early stopping, best iteration is:
[495]	valid_0's rmse: 72988.5
--------------------------------------------------
Fold 1 predictions: [20346.713 75038.54  49000.71  ... 22248.035 15387.039 45555.387]
[LightGBM] [Warning] bagging_fraction is set=0.5613792799670121, subsample=0.626110627013553 will be ignored. Current value: bagging_fraction=0.5613792799670121
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 wil

[I 2024-09-12 20:43:19,230] Trial 1 finished with value: 75444.79554524823 and parameters: {'num_leaves': 455, 'max_depth': 7, 'learning_rate': 0.005786157015815844, 'subsample': 0.626110627013553, 'colsample_bytree': 0.8230450705704881, 'reg_alpha': 5.527343001638237, 'reg_lambda': 1.2628609181565513e-05, 'min_data_in_leaf': 88, 'bagging_fraction': 0.5613792799670121, 'bagging_freq': 4}. Best is trial 0 with value: 75341.25196857267.


...

[LightGBM] [Warning] bagging_fraction is set=0.6001397250134362, subsample=0.6648598048068471 will be ignored. Current value: bagging_fraction=0.6001397250134362
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 78572
[300]	valid_0's rmse: 78124.7
[450]	valid_0's rmse: 77714
[600]	valid_0's rmse: 77326.5
[750]	valid_0's rmse: 76965.4
[900]	valid_0's rmse: 76631.8
[1050]	valid_0's rmse: 76326.6
[1200]	valid_0's rmse: 76045.8
[1350]	valid_0's rmse: 75781.2
[1500]	valid_0's rmse: 75533.2
[1650]	valid_0's rmse: 75310.7
[1800]	valid_0's rmse: 75104.3
[1950]	valid_0's rmse: 74915.5
[2100]	valid_0's rmse: 74737.8
[2250]	valid_0's rmse: 74574.4
[2400]	valid_0's rmse: 74422.6
[2550]	valid_0's rmse: 74280.8
[2700]	valid_0's rmse: 74154.

[I 2024-09-13 03:18:47,630] Trial 48 finished with value: 75241.87026033126 and parameters: {'num_leaves': 315, 'max_depth': 21, 'learning_rate': 0.00030915263744183796, 'subsample': 0.6648598048068471, 'colsample_bytree': 0.648729965025791, 'reg_alpha': 0.24426162033366014, 'reg_lambda': 4.9546108298729274e-05, 'min_data_in_leaf': 10, 'bagging_fraction': 0.6001397250134362, 'bagging_freq': 3}. Best is trial 42 with value: 75176.92035281133.


[LightGBM] [Warning] bagging_fraction is set=0.7980556929233544, subsample=0.6346522428924753 will be ignored. Current value: bagging_fraction=0.7980556929233544
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=143, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=143
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 77353.8
[300]	valid_0's rmse: 76090.8
[450]	valid_0's rmse: 75186.1
[600]	valid_0's rmse: 74518
[750]	valid_0's rmse: 74016.7
[900]	valid_0's rmse: 73654
[1050]	valid_0's rmse: 73392.7
[1200]	valid_0's rmse: 73208.9
[1350]	valid_0's rmse: 73073.9
[1500]	valid_0's rmse: 72975.9
[1650]	valid_0's rmse: 72909.9
[1800]	valid_0's rmse: 72865.2
[1950]	valid_0's rmse: 72833.9
[2100]	valid_0's rmse: 72817
[2250]	valid_0's rmse: 72806.5
[2400]	valid_0's rmse: 72805.6
[2550]	valid_0's rmse: 72807.1
Early stopping, best iteratio

[I 2024-09-13 03:25:42,633] Trial 49 finished with value: 75264.94265350817 and parameters: {'num_leaves': 405, 'max_depth': 19, 'learning_rate': 0.0012817037613122517, 'subsample': 0.6346522428924753, 'colsample_bytree': 0.5740518831095309, 'reg_alpha': 1.4566304317653694e-05, 'reg_lambda': 0.0001356127148622621, 'min_data_in_leaf': 143, 'bagging_fraction': 0.7980556929233544, 'bagging_freq': 5}. Best is trial 42 with value: 75176.92035281133.


{'num_leaves': 319, 'max_depth': 22, 'learning_rate': 0.0007005134877356104, 'subsample': 0.6027217911727021, 'colsample_bytree': 0.5837587354716537, 'reg_alpha': 0.26253273741141203, 'reg_lambda': 4.0754621086069554e-05, 'min_data_in_leaf': 19, 'bagging_fraction': 0.6418684277912554, 'bagging_freq': 5}


In [27]:
print(study.best_params)
best_params = study.best_params

{'num_leaves': 319, 'max_depth': 22, 'learning_rate': 0.0007005134877356104, 'subsample': 0.6027217911727021, 'colsample_bytree': 0.5837587354716537, 'reg_alpha': 0.26253273741141203, 'reg_lambda': 4.0754621086069554e-05, 'min_data_in_leaf': 19, 'bagging_fraction': 0.6418684277912554, 'bagging_freq': 5}


In [31]:

# lgb_params = {
#                 'num_leaves': 426,
#                  'max_depth': 20,
#                  'learning_rate': 0.011353178352988012,
#                  'n_estimators': 10000,
#                  'metric': 'rmse',
#                  'subsample': 0.5772552201954328,
#                  'colsample_bytree': 0.9164865430101521,
#                  'reg_alpha': 1.48699088003429e-06,
#                  'reg_lambda': 0.41539458543414265,
#                  'min_data_in_leaf': 73,
#                  'feature_fraction': 0.751673655170548,
#                  'bagging_fraction': 0.5120415391590843,
#                  'bagging_freq': 2,
#                  'random_state': 42,
#                  'min_child_weight': 0.017236362383443497,
#                  'cat_smooth': 54.81317407769262,
#                  'verbose' : -1            
# }

# lgb_params = {
#                 'num_leaves': 500,
#                  'max_depth': 20,
#                  'learning_rate': 0.011353178352988012,
#                  'n_estimators': 10000,
#                  'metric': 'rmse',
#                  'subsample': 0.5772552201954328,
#                  'colsample_bytree': 0.9164865430101521,
#                  'reg_alpha': 1.48699088003429e-06,
#                  'reg_lambda': 0.41539458543414265,
#                  'min_data_in_leaf': 73,
#                  'feature_fraction': 0.751673655170548,
#                  'bagging_fraction': 0.5120415391590843,
#                  'bagging_freq': 2,
#                  'random_state': 42,
#                  'min_child_weight': 0.017236362383443497,
#                  'cat_smooth': 54.81317407769262,
#                  'verbose' : -1            
# }
lgb_params = {
                'num_leaves': 500,
                 'max_depth': 20,
                 'learning_rate': 0.01,
                 'n_estimators': 10000,
                 'metric': 'rmse',
                 'subsample': 0.5857767336403017,
                 'colsample_bytree': 0.9164865430101521,
                 'reg_alpha': 1.32689078002428e-06,
                 'reg_lambda': 0.42348458521414265,
                 'min_data_in_leaf': 91,
                 'feature_fraction': 0.751673655170548,
                 'bagging_fraction': 0.5150827786006247,
                 'bagging_freq': 2,
                 'random_state': 42,
                 'min_child_weight': 0.017236362383443497,
                 'cat_smooth': 54.81317407769262,
                 'verbose' : -1            
}
# lgb_params = {
#                 'num_leaves': 233,
#                  'max_depth': 21,
#                  'learning_rate': 0.0037774133279232934,
#                  'n_estimators': 10000,
#                  'metric': 'rmse',
#                  'subsample': 0.5857767336403017,
#                  'colsample_bytree': 0.5254971772934024,
#                  'reg_alpha': 0.032872185314645044,
#                  'reg_lambda': 0.002053844414719716,
#                  'min_data_in_leaf': 97,
#                  'feature_fraction': 0.751673655170548,
#                  'bagging_fraction': 0.5150827786006247,
#                  'bagging_freq': 2,
#                  'random_state': 42,
#                  'min_child_weight': 0.017236362383443497,
#                  'cat_smooth': 54.81317407769262,
#                  'verbose' : -1            
# }

# {'num_leaves': 319, 
#  'max_depth': 22, 
#  'learning_rate': 0.0007005134877356104, 
#  'subsample': 0.6027217911727021, 
#  'colsample_bytree': 0.5837587354716537, 
#  'reg_alpha': 0.26253273741141203, 
#  'reg_lambda': 4.0754621086069554e-05, 
#  'min_data_in_leaf': 19, 
#  'bagging_fraction': 0.6418684277912554, 
#  'bagging_freq': 5}

# print('LightGBM Cross-Validation Results:\n')
# lgb_model,test_predsl = cross_validate_model_l(LGBMRegressor, X_train_full, y_train_full, X_test, lgb_params)

In [32]:
# 예를 들어 Optuna를 통해 얻은 최적의 하이퍼파라미터
# best_params = study.best_params
# print(best_params)

# # 추가로 고정해야 할 파라미터가 있다면 추가
# best_params['n_estimators'] = 10000  # 필요에 따라 수정
# best_params['metric'] = 'rmse'
# best_params['random_state'] = 42

# 교차 검증 실행
print('LightGBM Cross-Validation Results with Tuned Hyperparameters:\n')
lgb_model, test_preds = cross_validate_model_l(LGBMRegressor, X_train, y_train, X_test, lgb_params, n_splits=5)

# # 테스트 데이터에 대한 최종 예측 확인
print("Final Test Predictions:", test_preds[:10]) 

LightGBM Cross-Validation Results with Tuned Hyperparameters:

[LightGBM] [Warning] feature_fraction is set=0.751673655170548, colsample_bytree=0.9164865430101521 will be ignored. Current value: feature_fraction=0.751673655170548
[LightGBM] [Warning] bagging_fraction is set=0.5150827786006247, subsample=0.5857767336403017 will be ignored. Current value: bagging_fraction=0.5150827786006247
[LightGBM] [Warning] min_data_in_leaf is set=91, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=91
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 75581.4
[300]	valid_0's rmse: 75155.4
[450]	valid_0's rmse: 75182.5
Early stopping, best iteration is:
[352]	valid_0's rmse: 75140.9
--------------------------------------------------
Fold 1 predictions: [19145.83  72527.58  52304.9   ... 19505.158 14843.304 40280.28 ]
[LightGBM] [Warning] fea

In [33]:
lgb_result =  pd.read_csv('/home/baebro/kaggle_ws/kaggle_competitions/Regression of Used Car Prices/data/sample_submission.csv')
lgb_result['price'] = test_preds.astype(np.float32)
#xgb_result.to_csv('result_xgb.csv', index=False)
lgb_result
lgb_result.to_csv('submission_lgb_w_original_Optuna_tunning4.csv', index=False)